In [1]:
# Plotting notebook for the project
# Imports
import argparse
import os
import sys
import glob
import re

# Third-party imports # test
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
from datetime import datetime
import scipy.stats as stats
import dask.array as da
import dask
import dask.distributed as dd

# Import dask gateway
import dask_gateway

# import cdo
from cdo import *
cdo = Cdo()

# Local imports
sys.path.append('/home/users/benhutch/skill-maps-historical')
import dictionaries as dic
import functions as fnc

In [2]:
# Set up the parameters again
# for the years 2-9 psl ULG (JJA)
var = "psl"
region = "global"
region_grid = dic.gridspec_global
forecast_range = "2-9"
season = "DJFM" # weird season name for model
observations_path = dic.obs
obs_var_name = "psl"
model_dict = dic.model_dictionary_psl_historical_badc
start_year = 1960
end_year = 2014

In [3]:
# # Process the observations first of all
# obs = fnc.process_observations(var, region, region_grid,
#                                forecast_range, season, observations_path,
#                                obs_var_name)

In [4]:
# # First we want to merge the time axis and regrid the model (historical) data
# # using the function call_mergetime_regrid
# # this does not return anything, but saves the regridded data to a netcdf file
# fnc.call_mergetime_regrid(model_dict, var, region)

In [5]:
# Now we want to load the historical data
# as a dictionary of xarray datasets for each model
# using the load_historical_data function
historical_data = fnc.load_historical_data(model_dict, var, region)

# historical_data

processing model:  BCC-CSM2-MR
type of var <class 'str'>
type of model <class 'str'>
type of region <class 'str'>
regrid_files: /gws/nopw/j04/canari/users/benhutch/historical/psl/BCC-CSM2-MR/regrid/psl_Amon_BCC-CSM2-MR_historical_r*i?p?f?_*global_regrid.nc
number of files for model  BCC-CSM2-MR :  3
processing model:  MPI-ESM1-2-HR
type of var <class 'str'>
type of model <class 'str'>
type of region <class 'str'>
regrid_files: /gws/nopw/j04/canari/users/benhutch/historical/psl/MPI-ESM1-2-HR/regrid/psl_Amon_MPI-ESM1-2-HR_historical_r*i?p?f?_*global_regrid.nc
number of files for model  MPI-ESM1-2-HR :  1
processing model:  CanESM5
type of var <class 'str'>
type of model <class 'str'>
type of region <class 'str'>
regrid_files: /gws/nopw/j04/canari/users/benhutch/historical/psl/CanESM5/regrid/psl_Amon_CanESM5_historical_r*i?p?f?_*global_regrid.nc
number of files for model  CanESM5 :  35
processing model:  CMCC-CM2-SR5
type of var <class 'str'>
type of model <class 'str'>
type of region <cl

In [ ]:
# test = historical_data['NorCPM1'][0].psl

# # # test the processing functions individually
# constrained_data = fnc.constrain_historical_data_season(historical_data, start_year=1960, end_year=2019
#                                                         , season='DJFM', model='NorCPM1',member=0)

# constrained_data

# test the processing functions individually
# test_BCC = historical_data['BCC-CSM2-MR']

# # test the processing historical data function
processed_test_BCC = fnc.process_historical_data_dask(historical_data, season='DJFM', forecast_range='2-9', 
                                                start_year=1960, end_year=2019)

# look at the processed data
processed_test_BCC

processing model:  BCC-CSM2-MR
processing member:  0
processing member:  1
processing member:  2


In [ ]:
# # Look at the data we have processed
# processed_test_BCC['BCC-CSM2-MR'][2]

In [ ]:
# testing the ensemble mean
import xarray as xr

# Load the three datasets into a list
datasets = [historical_data['BCC-CSM2-MR'][member] for member in historical_data['BCC-CSM2-MR']]

# Combine the datasets into a single dataset along the 'member' dimension
ensemble = xr.concat(datasets, dim='member')

# Calculate the ensemble mean along the 'member' dimension
ensemble_mean = ensemble.mean(dim='member')

# Calculate the time mean along the 'time' dimension
time_mean = ensemble_mean.mean(dim='time')

time_mean.psl.values

In [ ]:
# Now call the parallel processing function
processed_historical_data = fnc.process_historical_data_parallel(historical_data,
                            season, forecast_range, start_year, end_year) 

In [ ]:
# now we want to call the function to process the historical data
# this will: test
# 1. constrain the data to the provided year range and season
# 2. calculate the climatology and remove this to create anomalies
# 3. calculate the annual mean anomalies from the monthly mean anomalies
# 4. calculate the running mean of these annual mean anomalies
# then add the processed data back into the dictionary
processed_historical_data = fnc.process_historical_data(historical_data, season, forecast_range, start_year, end_year)

In [ ]:
# Have a look at the processed data
processed_historical_data['NorCPM1'][0]

In [ ]:
# Now we want to process the historical data
# in preperation for calculating the spatial correlations
# this function constrains the years to only those available in all members
# and then calculates the equally weighted ensemble mean of all members
# using the function process_historical_data_spacial_correlations
ensemble_mean = fnc.process_historical_data_spatial_correlations(processed_historical_data)